In [1]:
import fiftyone as fo
from fiftyone import ViewField as F

# Load a small dataset
dataset = fo.zoo.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    max_samples=10,
)
print('hello')

***Python 3.8 Deprecation Notice***
Python 3.8 will no longer be supported in new releases after October 1, 2024.
Please upgrade to Python 3.9 or later.
For additional details please see https://deprecation.voxel51.com


Found annotations at '/home/lradovan/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient


***Python 3.8 Deprecation Notice***
Python 3.8 will no longer be supported in new releases after October 1, 2024.
Please upgrade to Python 3.9 or later.
For additional details please see https://deprecation.voxel51.com


Loading existing dataset 'coco-2017-validation-10'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [3]:
from libcom import ControlComModel
from libcom.utils.process_image import make_image_grid, draw_bbox_on_image
import cv2

# Helper function to get area of a bounding box
def get_box_area(box):
    # box format is [x, y, width, height]
    return box[2] * box[3]

fg_img = '/home/lradovan/workspace/libcom/tests/source/foreground/1.jpg'
mask = '/home/lradovan/workspace/libcom/tests/source/foreground_mask/1.png'

# Print info about each sample and its largest box
for sample in dataset[5:6]:
    # print(sample)
    detections = sample.ground_truth.detections
    if len(detections) > 0:
        print(f"\nImage path: {sample.filepath}")
        
        # Find largest bounding box
        largest_det = max(detections, key=lambda d: get_box_area(d.bounding_box))

        # get bbox
        bbox = largest_det.bounding_box
        width = sample.metadata.width
        height = sample.metadata.height
        
        x1 = int(bbox[0] * width)
        y1 = int(bbox[1] * height)
        x2 = int((bbox[0] + bbox[2]) * width)  # Corrected calculation for x2
        y2 = int((bbox[1] + bbox[3]) * height)

        # this 
        bbox = [x1, y1, x2, y2]

        occluded = largest_det.label
        bg_img = sample.filepath
        print(f"Largest object: {occluded}")
        print(f"Box coordinates: {bbox}")
        net     = ControlComModel(device=0)
        comp    = net(bg_img, fg_img, bbox, mask, task=['blending', 'harmonization'])
        bg_img  = draw_bbox_on_image(bg_img, bbox)
        grid_img = make_image_grid([bg_img, fg_img, comp[0], comp[1]])
        cv2.imwrite('./out.jpg', grid_img)


Image path: /home/lradovan/fiftyone/coco-2017/validation/data/000000000785.jpg
Largest object: person
Box coordinates: [280, 44, 499, 391]
